In [1]:
import pandas as pd
import numpy as np
import os,sys
import pickle
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import sklearn
from sklearn.utils import class_weight
from sklearn.decomposition import *
import string
import pyarrow
tf.__version__

'2.1.0'

In [2]:
 import sys,os
 print(sys.executable)
 print(sys.version)
 print(sys.version_info)
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()

#remote
# conf.setAppName("pcapAnalyzer")#pcap_analyzer pcapAnalyzer
# conf.setMaster("k8s://https://192.168.50.123:6443")
# conf.set("spark.driver.host", "192.168.50.123")
# conf.set("spark.executor.instances","2") #2
# conf.set("spark.executor.cores","4")
# conf.set("spark.driver.memory","40g")
# conf.set("spark.kubernetes.container.image", "smileocean/spark-py:spark_3.0.0_preview2_python")
# conf.set("spark.kubernetes.namespace", "spark-employee") #spark-employee
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName","spark-employee")#spark-employee
# conf.set("spark.kubernetes.pyspark.pythonVersion","3")

# conf.setMaster("k8s://https://192.168.50.123:6443")
# conf.set("spark.driver.host", "192.168.50.123")
# conf.set("spark.executor.cores", "3")
# conf.set("spark.executor.instances","2")
# conf.set("spark.executor.memory", "25g")
# conf.set("spark.kubernetes.container.image", "smileocean/spark-py:spark_3.0.0_preview2_python")
# conf.set("spark.kubernetes.namespace", "spark-employee")
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName","spark-employee")
# conf.set("spark.kubernetes.pyspark.pythonVersion","3")
# spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate()
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
# conf.set("spark.driver.extraJavaOptions","-Xss2048m")
# conf.set("spark.sql.shuffle.partitions", 1000)
#local
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","50g")
conf.set("spark.driver.extraJavaOptions","-Xss20480m")
# conf.set("spark.memory.storageFraction","0.2")
# conf.set("spark.driver.maxResultSize","0")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions", 20000)
conf.set("spark.driver.maxResultSize", "5g")

# spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringFields", 50000)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",20*1024*1024*1024)

import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
spark.version

/home/antslab/anaconda3/bin/python
3.7.7 (default, Mar 26 2020, 15:48:22) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
0.31.0


'3.0.0-preview2'

In [3]:
# df = ks.read_parquet('../../data_hdd4t/pcap_process/pcap_spark/2020_01_09/中華電信/parquet/2020_01_09_country_session.parquet/')
df = pd.read_parquet('../../data_hdd4t/pcap_process/pcap_spark/2020_01_09/中華電信/parquet/2020_01_09_country_session.parquet/')
#debug
# df = kf
# df = pickle.load(open('../../spark_data/data/Leo_Spark_Home/tmp/part_df.pkl','rb'))
# df

In [3]:
df

,session_lifetime,session_packets,tcp.window_size_value.avg,tcp.srcport,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,...,frame.protocols.common,frame.len.sum,city,subdivisions,latitude,longitude,domain,isp,network,country
0,0.000259,2,7300.000000,10553.0,8080.0,False,False,False,False,True,...,eth:ethertype:ip:tcp,114,None,None,33.0000,65.0000,None,Easy Connect- ISP,103.46.208.0/22,Afghanistan
1,0.591697,3,741.333333,42326.0,1433.0,False,False,False,False,True,...,eth:ethertype:ip:tcp,168,Kabul,Kabul,34.5167,69.1833,io-global.com,Io Global Services Pvt. Limited,125.213.192.0/19,Afghanistan
2,0.531377,3,741.333333,42326.0,1433.0,False,False,False,False,True,...,eth:ethertype:ip:tcp,168,Kabul,Kabul,34.5167,69.1833,io-global.com,Io Global Services Pvt. Limited,125.213.192.0/19,Afghanistan
3,0.000235,2,512.000000,43610.0,1433.0,False,False,False,False,True,...,eth:ethertype:ip:tcp,114,Kabul,Kabul,34.5167,69.1833,None,Unique Atlantic Telecommunication,103.18.160.0/22,Afghanistan
4,5.752291,17,39235.352941,42326.0,1433.0,False,False,False,False,True,...,eth:ethertype:ip:tcp,1256,Kabul,Kabul,34.5167,69.1833,io-global.com,Io Global Services Pvt. Limited,125.213.192.0/19,Afghanistan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5538123,30.411789,6,30845.000000,48733.0,54995.0,True,False,False,False,False,...,eth:ethertype:ip:tcp,402,None,None,47.0000,8.0000,telia.net,Telia Company,62.115.128.0/18,__HIVE_DEFAULT_PARTITION__
5538124,30.673302,10,30845.000000,51524.0,60384.0,True,False,False,False,False,...,eth:ethertype:ip:tcp,670,None,None,47.0000,8.0000,telia.net,Telia Company,62.115.128.0/18,__HIVE_DEFAULT_PARTITION__
5538125,0.838294,6,30845.000000,52305.0,57767.0,True,False,False,False,False,...,eth:ethertype:ip:tcp,402,None,None,47.0000,8.0000,telia.net,Telia Company,62.115.128.0/18,__HIVE_DEFAULT_PARTITION__
5538126,15.466875,4,30845.000000,62803.0,62086.0,True,False,False,False,False,...,eth:ethertype:ip:tcp,268,None,None,47.0000,8.0000,telia.net,Telia Company,62.115.128.0/18,__HIVE_DEFAULT_PARTITION__


In [4]:
df.columns

Index(['session_lifetime', 'session_packets', 'tcp.window_size_value.avg',
       'tcp.srcport', 'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin',
       'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'ip.src',
       'ip.dst', 'ip.ttl.avg', 'ip.proto', 'frame.protocols.common',
       'frame.len.sum', 'city', 'subdivisions', 'latitude', 'longitude',
       'domain', 'isp', 'network', 'country'],
      dtype='object')

### frame.protocols

In [5]:
df2 = df[['frame.protocols.common']]
df2

,frame.protocols.common
0,eth:ethertype:ip:tcp
1,eth:ethertype:ip:tcp
2,eth:ethertype:ip:tcp
3,eth:ethertype:ip:tcp
4,eth:ethertype:ip:tcp
...,...
5538123,eth:ethertype:ip:tcp
5538124,eth:ethertype:ip:tcp
5538125,eth:ethertype:ip:tcp
5538126,eth:ethertype:ip:tcp


In [6]:
def protocol_convert(s):
    '''
    s:
    '''
    ss = s.split(":")[3:]
    de_re = []
    last = None
    for sss in ss:
        if sss != last:
            de_re.append(sss)
            last = sss
    return " ".join(de_re)

In [7]:
df2['protocols'] = df2['frame.protocols.common'].map(protocol_convert)
all_pro = df2['protocols'].unique()
# all_pro = sorted(list(all_pro))
print('subtype categories#:',len(all_pro))
all_terms = []
length = []
for pro in all_pro:
    all_terms.extend(pro.split())
    length.append(len(pro.split()))
all_terms = sorted(list(set(all_terms)))
print("terms categories#:",len(all_terms))
all_dict = {}
for i,v in enumerate(all_terms):
    all_dict[v]=i+1
print("Protocol Max length =",max(length))
all_dict

subtype categories#: 163
terms categories#: 91
Protocol Max length = 7


/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'asap': 1,
 'ayiya': 2,
 'bacapp': 3,
 'bacnet': 4,
 'bfd': 5,
 'bvlc': 6,
 'c': 7,
 'chargen': 8,
 'cigi': 9,
 'cldap': 10,
 'cnip': 11,
 'coap': 12,
 'd': 13,
 'data': 14,
 'data-text-lines': 15,
 'dhcpv6': 16,
 'dns': 17,
 'dtls': 18,
 'e': 19,
 'echo': 20,
 'enip': 21,
 'enip_io': 22,
 'esp': 23,
 'gre': 24,
 'gss-api': 25,
 'gtp': 26,
 'gtpprime': 27,
 'h225': 28,
 'hart_ip': 29,
 'http': 30,
 'icmp': 31,
 'infiniband': 32,
 'ip': 33,
 'ipmi_session': 34,
 'ipv6': 35,
 'ipx': 36,
 'isakmp': 37,
 'kerberos': 38,
 'knx': 39,
 'l2tp': 40,
 'llc': 41,
 'lmp': 42,
 'm': 43,
 'mdns': 44,
 'memcache': 45,
 'mgcp': 46,
 'mpls': 47,
 'msproxy': 48,
 'nbns': 49,
 'nbss': 50,
 'ncp': 51,
 'ns_cert_exts': 52,
 'ntlmssp': 53,
 'ntp': 54,
 'omron': 55,
 'openvpn': 56,
 'pn_io': 57,
 'pn_rt': 58,
 'pop': 59,
 'portcontrol': 60,
 'ppp': 61,
 'quake3': 62,
 'r': 63,
 'raknet': 64,
 'rip': 65,
 'rmcp': 66,
 'rpc': 67,
 'rsip': 68,
 'rtcp': 69,
 's': 70,
 'scop': 71,
 'sdp': 72,
 'sip': 73,
 'smb':

In [8]:
def protocols_encode(s):
    '''
    s: df.protocol (Series)
    '''
    s_ = s.split()
    if len(s_)>1:
        temp = []
        for ss in s_:
            temp.append(all_dict[ss])
        return temp[1:]
    else:
        return [all_dict[s_[0]]]

In [9]:
df2['protocols_encode'] = df2.protocols.map(protocols_encode)
all_pro = df2.protocols_encode.tolist()
df2

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,frame.protocols.common,protocols,protocols_encode
0,eth:ethertype:ip:tcp,tcp,[83]
1,eth:ethertype:ip:tcp,tcp,[83]
2,eth:ethertype:ip:tcp,tcp,[83]
3,eth:ethertype:ip:tcp,tcp,[83]
4,eth:ethertype:ip:tcp,tcp,[83]
...,...,...,...
5538123,eth:ethertype:ip:tcp,tcp,[83]
5538124,eth:ethertype:ip:tcp,tcp,[83]
5538125,eth:ethertype:ip:tcp,tcp,[83]
5538126,eth:ethertype:ip:tcp,tcp,[83]


In [10]:
all_pro_np = tf.keras.preprocessing.sequence.pad_sequences(all_pro,dtype='int32', padding='post')
assert all_pro_np.shape[1] == max(length)-1
assert np.max(all_pro_np) == max(all_dict.values())
all_pro_np.shape #trainX

(5538128, 6)

In [11]:
def convert_pro_y(s):
    '''
    s: protocols (String series)
    0=>tcp
    1=>udp
    2=>icmp
    (sparse categorical cross entropy)
    '''
    if 'tcp' in s:
        return 0
    elif 'udp' in s:
        return 1
    elif 'icmp' in s:
        return 2
    else:
        return 'ERR'

In [12]:
df2['protocols_ans'] = df2.protocols.map(convert_pro_y)
all_pro_ans = df2['protocols_ans'].tolist()
class_weight_ = dict(Counter(all_pro_ans))
print(class_weight_)
all_pro_ans = np.array(all_pro_ans)
assert all_pro_ans.shape[0]==all_pro_np.shape[0]

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{0: 5387047, 1: 114815, 2: 36266}


In [74]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(all_pro_ans),
                                                 all_pro_ans)
class_weights

array([3.43288706e-01, 1.16144019e+01, 1.11111111e+03])

In [13]:
embed_dim = 4
class protol_model(Model):
    def __init__(self):
        super(protol_model,self).__init__()
        self.emb = Embedding(np.max(all_pro_np)+1,embed_dim,input_length=all_pro_np.shape[1],mask_zero=True)
        self.rnn = LSTM(embed_dim,recurrent_dropout=0.2,dropout=0.2)
        self.out = Dense(max(all_pro_ans)+1,activation='softmax')
    def call(self,x):
        x = self.emb(x)
        x = self.rnn(x)
        x = self.out(x)
        return x
model = protol_model()
# model(all_pro_np[:2])

In [15]:
loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# class_weights = tf.constant(list(class_weights))
# class_weights = tf.compat.v2.constant(list(class_weights))
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
input_shape=(None,all_pro_np.shape[1])
model.build(input_shape)
model.summary()

Model: "protol_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  368       
_________________________________________________________________
lstm (LSTM)                  multiple                  144       
_________________________________________________________________
dense (Dense)                multiple                  15        
Total params: 527
Trainable params: 527
Non-trainable params: 0
_________________________________________________________________


In [16]:
H = model.fit(all_pro_np, all_pro_ans, epochs=5) #batch size, epoch, save best, early stopping

Train on 5538128 samples
Epoch 1/5
 913888/5538128 [===>..........................] - ETA: 17:58 - loss: 0.5599 - accuracy: 0.9952

KeyboardInterrupt: 

In [ ]:
# sum all emb layer & lstm output to numpy shape=(N,4)

### isp

In [20]:
df3 = df.loc[:,'isp']
df3 = pd.DataFrame(df3)
df3 = df3.fillna('<UNK>')
df3

,isp
0,Easy Connect- ISP
1,Io Global Services Pvt. Limited
2,Io Global Services Pvt. Limited
3,Unique Atlantic Telecommunication
4,Io Global Services Pvt. Limited
...,...
5538123,Telia Company
5538124,Telia Company
5538125,Telia Company
5538126,Telia Company


In [21]:
def get_tokens(s):
    '''
    s: str with space split (isp)(series)
    return: list
    '''
    ss = s.translate(str.maketrans('', '', string.punctuation))
    return ss.lower()#.split()

In [22]:
df3['isp_tokens_li'] = df3.isp.map(get_tokens)
isp_tokens = df3['isp_tokens_li'].unique()
isp_tokens = sorted(list(set(" ".join(isp_tokens).split()))) #all dict
# isp_tokens = sum(isp_tokens,[])
isp_dict = {}
for i,v in enumerate(isp_tokens):
    isp_dict[v]=i+1
print(len(isp_dict))
isp_dict #save pkl?

{'013': 1,
 '015ltd': 2,
 '018': 3,
 '07internetro': 4,
 '099': 5,
 '1': 6,
 '102': 7,
 '10dencehispahard': 8,
 '11': 9,
 '123net': 10,
 '16x': 11,
 '1st': 12,
 '1telecom': 13,
 '1tennessee': 14,
 '1toc': 15,
 '2': 16,
 '2000': 17,
 '2002': 18,
 '2003': 19,
 '2007': 20,
 '21': 21,
 '23media': 22,
 '23netru': 23,
 '23vnet': 24,
 '24hk': 25,
 '263net': 26,
 '2bite': 27,
 '2com': 28,
 '2d': 29,
 '2day': 30,
 '2degrees': 31,
 '2f': 32,
 '3': 33,
 '31173': 34,
 '323325': 35,
 '335': 36,
 '360': 37,
 '3bb': 38,
 '3c1b': 39,
 '3d': 40,
 '3f': 41,
 '3g': 42,
 '3m': 43,
 '3play': 44,
 '3s': 45,
 '3w': 46,
 '3wlink': 47,
 '4': 48,
 '4inet': 49,
 '4th': 50,
 '5': 51,
 '51a': 52,
 '58': 53,
 '6p': 54,
 '7': 55,
 '702': 56,
 '76': 57,
 '77': 58,
 '78': 59,
 '7heaven': 60,
 '8bit': 61,
 '92cloud': 62,
 '9karu': 63,
 '9mobile': 64,
 'a': 65,
 'a1': 66,
 'a2': 67,
 'a3': 68,
 'aachen': 69,
 'aakash': 70,
 'aamra': 71,
 'aapt': 72,
 'aarnet': 73,
 'ab': 74,
 'abadi': 75,
 'abaete': 76,
 'abak': 77,
 'a

In [24]:
def encode_tokens(s):
    '''
    s: isp_tokens (str)
    '''
    tokens_int = []
    for token in s.split():
        tokens_int.append(isp_dict[token])
    return tokens_int

In [25]:
df3['isp_encode'] = df3.isp_tokens_li.map(encode_tokens) #轉換token
isp_encode_np = df3.isp_encode.tolist()
isp_encode_np = tf.keras.preprocessing.sequence.pad_sequences(isp_encode_np,dtype='int32', padding='post') #計算maxlen、padding
assert np.max(isp_encode_np) == max(isp_dict.values())
isp_encode_np.shape #trainX

(5538128, 11)

In [26]:
ori_dict = dict(Counter(df3.isp.tolist()))
max_num = max(ori_dict.values())
min_num = min(ori_dict.values())
range_ = max_num-min_num
new_dict = {}
for k,v in ori_dict.items():
    new_dict[k]=(v-min_num)/(range_)
print(len(new_dict))
new_dict    

{'Easy Connect- ISP': 7.961010156656758e-06,
 'Io Global Services Pvt. Limited': 0.00032958582048558976,
 'Unique Atlantic Telecommunication': 3.343624265795838e-05,
 'Sanzar F Kakar T/A Afghanistan Holding Group': 4.776606093994055e-05,
 'Afghantelecom Government Communication Network': 1.1145414219319461e-05,
 'Etisalat Afghanistan': 7.961010156656758e-06,
 'Afghan Cyber ISP': 0.0,
 'Network Zone Solution Provider': 6.368808125325406e-06,
 'Etisalat Afghan': 1.5922020313313515e-06,
 'Instatelecom Limited': 6.368808125325406e-06,
 'Ankabut Internet Service Provider': 3.184404062662703e-06,
 'Afghan Wireless': 1.5922020313313515e-06,
 'Cyber Telecom ISP': 0.0,
 'Wireless, Dial-Up Internet Service Provider': 0.0,
 'AFSAT': 0.0,
 'North Telecom': 1.5922020313313515e-06,
 'Keminet Ltd.': 0.0005413486906526595,
 'Kujtesa Net Sh.p.k.': 0.00033277022454825247,
 'IPKO Telecommunications LLC': 0.0006496184287831915,
 'Digicom': 1.7514222344644868e-05,
 'Abissnet sh.a.': 0.0001974330518850876,


In [27]:
df3['isp_ans'] = df3.isp.map(new_dict)#計算各isp詞頻答案
isp_ans_np = df3.isp_ans.to_numpy()
assert isp_ans_np.shape[0]==isp_encode_np.shape[0]
isp_ans_np.shape

(5538128,)

In [41]:
embed_dim = 3
class isp_model(Model):
    def __init__(self):
        super(isp_model,self).__init__()
        self.emb = Embedding(np.max(isp_encode_np)+1,embed_dim,mask_zero=True,
                             input_length=isp_encode_np.shape[1])
        self.rnn = GRU(embed_dim,recurrent_dropout=0.2,dropout=0.2)
        self.out = Dense(1)
    def call(self,x):
        x = self.emb(x)
        x1 = self.rnn(x)
        x = self.out(x1)
        return x
model = isp_model()

model(isp_encode_np[:2])

#build model(embedding layer, rnn ,dense)=>regression

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00098642],
       [0.01209094]], dtype=float32)>

In [43]:
model.compile(optimizer=tf.keras.optimizers.Nadam(), loss='mse', metrics=['mae'])
model.summary()

Model: "isp_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  31053     
_________________________________________________________________
gru_3 (GRU)                  multiple                  72        
_________________________________________________________________
dense_4 (Dense)              multiple                  4         
Total params: 31,129
Trainable params: 31,129
Non-trainable params: 0
_________________________________________________________________


In [44]:
H = model.fit(isp_encode_np,isp_ans_np,epochs=6,batch_size=512)#adam 128/256? nadam 128/256?

Train on 5538128 samples
Epoch 1/6
5538128/5538128 [==============================] - 144s 26us/sample - loss: 0.0047 - mae: 0.0329
Epoch 2/6
5538128/5538128 [==============================] - 142s 26us/sample - loss: 4.4172e-04 - mae: 0.0127
Epoch 3/6
5538128/5538128 [==============================] - 142s 26us/sample - loss: 3.4605e-04 - mae: 0.0100
Epoch 4/6
 254464/5538128 [>.............................] - ETA: 2:15 - loss: 3.1833e-04 - mae: 0.0093

KeyboardInterrupt: 

### Country、City、Subdivisions
* 計算各欄位NAN的次數，把低於這個次數的token也濾掉統一變成<UNK>
* kf寫成applymap function全部一起做掉?

In [4]:
def convert_columns(x):
    x = str(x).lower()
    return x.translate(str.maketrans('', '', string.punctuation))

In [7]:
kf = ks.read_parquet('../../data_hdd4t/pcap_process/pcap_spark/2020_01_09/中華電信/parquet/2020_01_09_country_session.parquet/')
df4 = kf[['country','city','subdivisions']]#.astype('string')
# punct = string.punctuation #'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'
# transtab = str.maketrans(dict.fromkeys(punct, ''))
# df4['country'] = df4['country'].str.lower()
# df4['country'] = df4['country'].str.translate(transtab)#.lower()
# df4['city'] = df4['city'].str.lower()
# df4['city'] = df4['city'].str.translate(transtab)#.lower()
# df4['subdivisions'] = df4['subdivisions'].str.lower()
# df4['subdivisions'] = df4['subdivisions'].str.translate(transtab)#.lower()
df4 = df4.fillna('nan')
df4 = df4.applymap(convert_columns)
df4.head()

,country,city,subdivisions
0,dominican republic,santo domingo este,provincia de santo domingo
1,dominican republic,santo domingo este,provincia de santo domingo
2,dominican republic,santo domingo este,provincia de santo domingo
3,dominican republic,santo domingo este,provincia de santo domingo
4,dominican republic,santo domingo este,provincia de santo domingo


In [11]:

transtab = str.maketrans(dict.fromkeys(punct, ''))
df['text'] = '|'.join(df['text'].tolist()).translate(transtab).split('|')

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
df4 = ks.get_dummies(df4,prefix=list(df4.columns))
df4.head()

KeyboardInterrupt: 

In [ ]:
df_encode_np = df4.to_numpy()
df_encode_np.shape

In [ ]:
pca = PCA(n_components=3,iterated_power='auto', whiten=True,svd_solver="full") #n_components='mle',395
encode_npy = pca.fit_transform(df_encode.to_numpy())

In [5]:
df4 = df[['country','city','subdivisions']].astype("category")
df4['country'] = df4.country.cat.add_categories('nan')
df4['city'] = df4.city.cat.add_categories('nan')
df4['subdivisions'] = df4.subdivisions.cat.add_categories('nan')
df4 = df4.fillna('nan')
df4 = df4.applymap(convert_columns)
df4
# df4 = df4.fillna('<UNK>')
# df4, dummy_na=True

,country,city,subdivisions
0,afghanistan,nan,nan
1,afghanistan,kabul,kabul
2,afghanistan,kabul,kabul
3,afghanistan,kabul,kabul
4,afghanistan,kabul,kabul
...,...,...,...
5538123,hivedefaultpartition,nan,nan
5538124,hivedefaultpartition,nan,nan
5538125,hivedefaultpartition,nan,nan
5538126,hivedefaultpartition,nan,nan


In [6]:
dict1 = dict(Counter(df4['country'].tolist()))
len1 = list(dict1.values())
dict1 = sorted(dict1.items(), key=lambda x: x[1])

# print(len(country_bad))

dict2 = dict(Counter(df4['city'].tolist()))
len2 = list(dict2.values())
dict2 = sorted(dict2.items(), key=lambda x: x[1])

# print(len(city_bad))

dict3 = dict(Counter(df4['subdivisions'].tolist()))
len3 = list(dict3.values())
dict3 = sorted(dict3.items(), key=lambda x: x[1])

# print(len(sub_bad))
dict3

[('kandahar', 1),
 ('souk ahras', 1),
 ('tébessa', 1),
 ('constantine', 1),
 ('khenchela', 1),
 ('ouargla', 1),
 ('boumerdes', 1),
 ('medea', 1),
 ('blida', 1),
 ('lori', 1),
 ('west grand bahama district', 1),
 ('northern', 1),
 ('noakhali', 1),
 ('chandpur', 1),
 ('madaripur', 1),
 ('natore', 1),
 ('habiganj', 1),
 ('christ church', 1),
 ('mono', 1),
 ('trashigang dzongkhag', 1),
 ('northwest', 1),
 ('praia', 1),
 ('tboung khmum', 1),
 ('ouham', 1),
 ('hadjerlamis', 1),
 ('departamento de la guajira', 1),
 ('tshuapa', 1),
 ('bascongo', 1),
 ('okres plzensever', 1),
 ('okres brnovenkov', 1),
 ('blansko district', 1),
 ('okres usti nad labem', 1),
 ('okres prahavychod', 1),
 ('rokycany district', 1),
 ('klatovy district', 1),
 ('jičín district', 1),
 ('provincia de peravia', 1),
 ('provincia del carchi', 1),
 ('provincia de santo domingo de los tsachilas', 1),
 ('bioko norte', 1),
 ('kiéntem', 1),
 ('southern nations nationalities and peoples region', 1),
 ('upper corsica', 1),
 ('haut

In [7]:
def knee_point(length_li,k=15):
    """
    length_li: 長度的list
    k: slope topK's knee points
    Return dict format (key is the answer)
    """
    length_dict = dict(Counter(length_li))
    sorted_dict = {k: v for k, v in sorted(length_dict.items(), key=lambda x: x[1])}
    all_items_num = sum(list(length_dict.values()))
#     for item in sorted_dict.items():
    all_keys = list(sorted_dict.keys())
    all_values = list(sorted_dict.values())
    slope_li = []
    for i in range(len(sorted_dict)):
        length1 = all_keys[i]
        try:
            length2 = all_keys[i+1]
        except IndexError:
            break
        value1 = sum(all_values[:i+1])
        value2 = sum(all_values[:i+2])
        slope = ((value2-value1)/all_items_num)/(length2-length1)
        slope_li.append(slope)
    change_rate_li = []
    for i in range(len(slope_li)):
        try:
            slope1 = slope_li[i]
            slope2 = slope_li[i+1]
        except IndexError:
            break
        change_rate_li.append(abs(slope2-slope1)) #陡變緩或是緩變陡的都一起算
    idx_li = sorted(range(len(change_rate_li)), key=lambda i: change_rate_li[i], reverse=True)[:k]
    return_dict = {}
    for idx in idx_li:
        return_dict[all_keys[idx+1]] = change_rate_li[idx]
    return pd.DataFrame.from_dict(return_dict,orient='index',columns=['knee_slope'])

In [8]:
temp1= knee_point(len1) #4 11 12 41
temp2= knee_point(len2) #2 3 4 13
temp3= knee_point(len3) #6 9 10 14
print(temp1)
print(temp2)
temp3

     knee_slope
4      0.041667
2      0.020882
3      0.010000
11     0.005010
12     0.004968
8      0.002526
10     0.002499
41     0.001063
382    0.000646
215    0.000565
224    0.000555
367    0.000364
28     0.000342
366    0.000295
7      0.000294
    knee_slope
2     0.084567
3     0.041038
4     0.028946
13    0.020702
12    0.017441
14    0.016342
5     0.016195
15    0.015462
19    0.012348
18    0.011872
11    0.008904
21    0.008794
7     0.006449
6     0.006156
17    0.005533


,knee_slope
6,0.038163
5,0.031224
9,0.023878
4,0.022653
2,0.017959
3,0.017551
8,0.017483
10,0.010204
14,0.008912
15,0.008776


In [9]:
country_bad = []
for k,v in dict1:
    if v<4: #改
        country_bad.append(k)
    else:
        print(len(dict1)-len(country_bad))
        break
city_bad = []
for k,v in dict2:
    if v<2: #改
        city_bad.append(k)
    else:
        print(len(dict2)-len(city_bad))
        break
sub_bad = []
for k,v in dict3:
    if v<6: #改
        sub_bad.append(k)
    else:
        print(len(dict3)-len(sub_bad))
        break        

186
10772
1911


In [10]:
def unk_func(s,l):
    '''
    s: string series
    l: bad list
    '''
    if s in l:
        return 'unk'
    else:
        return s

In [11]:
df4['country'] = df4.country.apply(unk_func, args=(set(country_bad),))
df4['city'] = df4.city.apply(unk_func, args=(set(city_bad),))
df4['subdivisions'] = df4.subdivisions.apply(unk_func, args=(set(sub_bad),))
df4.country.unique().shape, df4.city.unique().shape , df4.subdivisions.unique().shape

((187,), (10773,), (1912,))

In [12]:
kf4 = ks.from_pandas(df4)
kf_encode = ks.get_dummies(kf4,prefix=list(df4.columns))

In [ ]:
pca = PCA(n_components=3,iterated_power='auto', whiten=True,svd_solver="full") #n_components='mle',395
encode_npy = pca.fit_transform(kf_encode.to_numpy())
encode_npy.shape

/home/antslab/anaconda3/lib/python3.7/site-packages/databricks/koalas/generic.py:545: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))


In [ ]:
df_encode = pd.get_dummies(df4)
df_encode

In [60]:
df_country = pd.get_dummies(df4['country'])
df41 = df4.join(df_country)
len1 = len(df_country.columns)
df_city = pd.get_dummies(df41['city'])
df42 = df41.join(df_city, lsuffix='_left', rsuffix='_right')
len2 = len(df_city.columns)
df_sub = pd.get_dummies(df42['subdivisions'])
df43 = df42.join(df_sub, lsuffix='_left', rsuffix='_right')
len3 = len(df_sub.columns)
assert len1+len2+len3+3 == len(df43.columns)
df43

ValueError: columns overlap but no suffix specified: Index(['Angola', 'Armenia', 'Costa Rica', 'Jamaica', 'Lebanon', 'Liberia',
       'Macao', 'Monaco', 'Montenegro', 'Singapore', 'Sudan', '<UNK>'],
      dtype='object')

In [ ]:
df_encode = df43.iloc[:,3:]
df_encode = df_encode.to_numpy()
df_encode.shape

In [ ]:
pca = PCA(n_components=3,iterated_power='auto', whiten=True,svd_solver="full") #n_components='mle',395
encode_npy = pca.fit_transform(df_encode)
encode_npy.shape #final encoding result (city country subdivision)

### domain

In [209]:
df5 = df.loc[:,'domain']
df5 = pd.DataFrame(df5)
df5 = df5.fillna('<NAN>')
df5

,domain
0,maxko-hosting.com
1,maxko-hosting.com
2,maxko-hosting.com
3,maxko-hosting.com
4,maxko-hosting.com
...,...
214748375605,magticom.ge
214748375606,magticom.ge
214748375607,magticom.ge
214748375608,magticom.ge


In [214]:
def convert_domain_li(s):
    '''
    s: domain series(str) split by "."
    '''
    return s.split('.')
def calc_length(s):
    '''
    calculate list's length in Series
    '''
    return len(s)
def prepend_li(s):
    '''
    prepend <NAN> to max_num
    '''
    ss = s[:]
    if len(ss)<max_num:
        iter_num = max_num-len(ss)
        for _ in range(iter_num):
            ss.insert(0,'<NAN>')
    return ss

In [215]:
df5['domain_li'] = df5.domain.map(convert_domain_li)
df5['domain_length'] = df5.domain_li.map(calc_length)
max_num = df5.domain_length.max()
print(max_num)
df5['domain_li_rev'] = df5.domain_li.map(prepend_li)
df5

3


,domain,domain_li,domain_length,domain_li_rev
0,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
1,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
2,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
3,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
4,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
...,...,...,...,...
214748375605,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375606,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375607,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375608,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"


In [216]:
col_name = []
for i in range(max_num):
    col_name.append('slot'+str(i+1))
df51 = pd.DataFrame(df5['domain_li_rev'].tolist(),columns=col_name)
df51

,slot1,slot2,slot3
0,<NAN>,maxko-hosting,com
1,<NAN>,maxko-hosting,com
2,<NAN>,maxko-hosting,com
3,<NAN>,maxko-hosting,com
4,<NAN>,maxko-hosting,com
...,...,...,...
99995,<NAN>,magticom,ge
99996,<NAN>,magticom,ge
99997,<NAN>,magticom,ge
99998,<NAN>,magticom,ge


In [219]:
for col in df51.columns:
    temp = dict(Counter(df51[col].tolist()))
    max_num = max(temp.values())
    min_num = min(temp.values())
    range_ = max_num-min_num
    def min_max_norm(s):
        ss = int(temp[s])
        ss = (ss-min_num)/range_
        return ss
    df51[col+'_enc'] = df51[col].map(min_max_norm)
df51    

,slot1,slot2,slot3,slot1_enc,slot2_enc,slot3_enc
0,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
1,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
2,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
3,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
4,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
...,...,...,...,...,...,...
99995,<NAN>,magticom,ge,1.0,1.000000,1.000000
99996,<NAN>,magticom,ge,1.0,1.000000,1.000000
99997,<NAN>,magticom,ge,1.0,1.000000,1.000000
99998,<NAN>,magticom,ge,1.0,1.000000,1.000000


In [223]:
col_li = df51.columns
col_li = [x for x in col_li if '_enc' in x]
domain_enc = df51[col_li].to_numpy()
domain_enc.shape #final encoding result (domain)

(100000, 3)

### tcp.dstport、tcp.srcport
* 要改attribute name

In [226]:
df6 = df[['tcp.dstport','tcp.srcport']]
df6 = df6.fillna(-1)
df6

,tcp.dstport,tcp.srcport
0,-1,-1
1,-1,-1
2,-1,-1
3,-1,-1
4,-1,-1
...,...,...
214748375605,[59305],[3389]
214748375606,[59304],[3389]
214748375607,[59332],[3389]
214748375608,[59317],[3389]


In [229]:
def convert_port(s):
    '''
    if -1: return [-1,-1,-1,-1]
    else minMax norm & category one-hot
    '''
    if s == -1:
        return [-1,-1,-1,-1]
    else:
        ss = int(s[0])
        minmax = [(ss-1)/65534]
        if ss <1024:
            minmax.extend([1,0,0])
        elif ss<49152:
            minmax.extend([0,1,0])
        else:
            minmax.extend([0,0,1])
        return minmax

In [231]:
df6['port1'] = df6['tcp.dstport'].map(convert_port)
df6['port2'] = df6['tcp.srcport'].map(convert_port)
df6['ports'] = df6['port1']+df6['port2']
df6

,tcp.dstport,tcp.srcport,port1,port2,ports
0,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
1,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
2,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
3,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
4,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
...,...,...,...,...,...
214748375605,[59305],[3389],"[0.9049348429822688, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9049348429822688, 0, 0, 1, 0.05169835505233..."
214748375606,[59304],[3389],"[0.9049195837275308, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9049195837275308, 0, 0, 1, 0.05169835505233..."
214748375607,[59332],[3389],"[0.9053468428601947, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9053468428601947, 0, 0, 1, 0.05169835505233..."
214748375608,[59317],[3389],"[0.9051179540391248, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9051179540391248, 0, 0, 1, 0.05169835505233..."


In [234]:
df61 = pd.DataFrame(df6['ports'].tolist(),columns=None)
ports_enc = df61.to_numpy()
ports_enc.shape  #final encoding result (2 ports)

(100000, 8)

In [ ]:
#flags、proto
#windows size、ttl、Frame.len可能要改名字
#network、ip.dst 、 ip.src 
#latitude、longtitude

#k-means: 群內距離短、群間距離大，取平均值找最佳的k
# cluster = KMeans(init='k-means++',n_init=10,max_iter=350,precompute_distances='auto',algorithm='auto',random_state=725035,n_clusters=2,n_jobs=-1,verbose=0) #11,305
# cluster.fit(train_X_PCA)

# ## 範例
# kmeans_fit = cluster.KMeans(n_clusters = 3).fit(iris_X)
# cluster_labels = kmeans_fit.labels_
# # 印出績效，選大值接近1者
# silhouette_avg = metrics.silhouette_score(iris_X, cluster_labels)
# print(silhouette_avg)
# plt.plot(range(2,20), silhouette_avg)